# py-coroutine

In [ ]:
'''gevent对协程的支持,本质上是greenlet在实现切换工作,'''

# gevent使用流程
# spawn方法可以看作是创建一个协程, joinall方法可以看作是添加任务,并启动运行协程
import requests
import gevent
from gevent import monkey
monkey.patch_all()


def run_task(url):
    try:
        r = requests.get(url)
    except Exception as e:
        print(e)

if __name__ == '__main__':
    urls = ['url1', 'url2', 'url3']
    greenlets = [gevent.spawn(run_task, url) for url in urls]
    gevent.joinall(greenlets)


# 协程池的使用
# -*- coding:utf-8 -*-
import gevent
from gevent import monkey
from gevent.pool import Pool
monkey.patch_all()
exist_pages = []
with open("xxxxx.txt", "r", encoding="utf-8") as f:
    for line in f:
        exist_pages.append(int(line.strip()))


def get_page_index():
    for page in range(1, 623000):
        yield page


def record_leak_pages(page):
    with open("xxxx.txt", "a", encoding="utf-8") as f:
        f.write("{}\n".format(page))


def check_page_exist(page):
    if page not in exist_pages:
        record_leak_pages(page)


pool = Pool(200)
try:
    pool.map(check_page_exist, get_page_index())
except Exception as e:
    print(e)


# asycio使用
# asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。
# asyncio的编程模型就是一个消息循环。我们从asyncio模块中直接获取一个EventLoop的引用，
# 然后把需要执行的协程扔到EventLoop中执行，就实现了异步IO。
# asycio执行一个任务
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
loop.close()

# 使用3.5版本async和await关键字
import asyncio
async def hello():
    print('hello,world')
    r=await asyncio.sleep(1)
    print('hello,again')

loop=asyncio.get_event_loop()
loop.run_until_complete(hello())
loop.close()

# asycio执行多个任务
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

# 使用asycio异步网络连接来获取sina、sohu和163的网站首页,并打印header内容

import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()